In [191]:
!pip install implicit

In [192]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [193]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Для работы с матрицами
from scipy.sparse import csr_matrix, coo_matrix

# Детерминированные алгоритмы
from implicit.nearest_neighbours import ItemItemRecommender, CosineRecommender, TFIDFRecommender, BM25Recommender

# Метрики
from implicit.evaluation import train_test_split
from implicit.evaluation import precision_at_k, mean_average_precision_at_k, AUC_at_k, ndcg_at_k

In [194]:
data = pd.read_csv('/content/drive/MyDrive/webinar_2/retail_train.csv')
data.head()

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.60,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.00,1631,1,0.0,0.0
2,2375,26984851472,1,1036325,1,0.99,364,-0.30,1631,1,0.0,0.0
3,2375,26984851472,1,1082185,1,1.21,364,0.00,1631,1,0.0,0.0
4,2375,26984851472,1,8160430,1,1.50,364,-0.39,1631,1,0.0,0.0


In [195]:
test_size_weeks = 3

data_train = data[data['week_no'] < data['week_no'].max() - test_size_weeks]
data_test = data[data['week_no'] >= data['week_no'].max() - test_size_weeks]

In [196]:
users, items, interactions = data.user_id.nunique(), data.item_id.nunique(), data.shape[0]

print('# users: ', users)
print('# items: ', items)
print('# interactions: ', interactions)

# users:  2499
# items:  89051
# interactions:  2396804


In [197]:
popularity = data.groupby('item_id')['sales_value'].sum().reset_index()
popularity.describe()

,item_id,sales_value
count,8.905100e+04,89051.000000
mean,5.115772e+06,83.458481
std,5.178973e+06,1628.715079
min,2.567100e+04,0.000000
25%,9.665830e+05,3.500000
50%,1.448516e+06,10.780000
75%,9.553042e+06,46.105000
max,1.802456e+07,467993.620000


In [198]:
popularity = data.groupby('item_id')['user_id'].nunique().reset_index()
popularity.describe()

,item_id,user_id
count,8.905100e+04,89051.000000
mean,5.115772e+06,14.759767
std,5.178973e+06,45.904111
min,2.567100e+04,1.000000
25%,9.665830e+05,1.000000
50%,1.448516e+06,2.000000
75%,9.553042e+06,10.000000
max,1.802456e+07,2039.000000


In [199]:
item_features = pd.read_csv('/content/drive/MyDrive/webinar_2/product.csv')
item_features.head()

,PRODUCT_ID,MANUFACTURER,DEPARTMENT,BRAND,COMMODITY_DESC,SUB_COMMODITY_DESC,CURR_SIZE_OF_PRODUCT
0,25671,2,GROCERY,National,FRZN ICE,ICE - CRUSHED/CUBED,22 LB
1,26081,2,MISC. TRANS.,National,NO COMMODITY DESCRIPTION,NO SUBCOMMODITY DESCRIPTION,
2,26093,69,PASTRY,Private,BREAD,BREAD:ITALIAN/FRENCH,
3,26190,69,GROCERY,Private,FRUIT - SHELF STABLE,APPLE SAUCE,50 OZ
4,26355,69,GROCERY,Private,COOKIES/CONES,SPECIALTY COOKIES,14 OZ


In [200]:
user_features = pd.read_csv('/content/drive/MyDrive/webinar_2/hh_demographic.csv')
user_features.head()

,AGE_DESC,MARITAL_STATUS_CODE,INCOME_DESC,HOMEOWNER_DESC,HH_COMP_DESC,HOUSEHOLD_SIZE_DESC,KID_CATEGORY_DESC,household_key
0,65+,A,35-49K,Homeowner,2 Adults No Kids,2,None/Unknown,1
1,45-54,A,50-74K,Homeowner,2 Adults No Kids,2,None/Unknown,7
2,25-34,U,25-34K,Unknown,2 Adults Kids,3,1,8
3,25-34,U,75-99K,Homeowner,2 Adults Kids,4,2,13
4,45-54,B,50-74K,Homeowner,Single Female,1,None/Unknown,16


In [201]:
test_users = result.shape[0]
new_test_users = len(set(data_test['user_id']) - set(data_train['user_id']))

print('В тестовом дата сете {} юзеров'.format(test_users))
print('В тестовом дата сете {} новых юзеров'.format(new_test_users))

В тестовом дата сете 2042 юзеров
В тестовом дата сете 0 новых юзеров


### Random recommendation

In [202]:
def random_recommendation(items, n=5):
    """Случайные рекоммендации"""
    
    items = np.array(items)
    recs = np.random.choice(items, size=n, replace=False)
    
    return recs.tolist()

In [203]:
%%time

items = data_train.item_id.unique()

result['random_recommendation'] = result['user_id'].apply(lambda x: random_recommendation(items, n=5))

CPU times: user 5.87 s, sys: 73.1 ms, total: 5.95 s
Wall time: 7.46 s


### Popularity-based recommendation

In [204]:
def popularity_recommendation(data, n=5):
    """Топ-n популярных товаров"""
    
    popular = data.groupby('item_id')['sales_value'].sum().reset_index()
    popular.sort_values('sales_value', ascending=False, inplace=True)
    
    recs = popular.head(n).item_id
    
    return recs.tolist()

In [205]:
%%time

# Можно так делать, так как рекомендация не зависит от юзера
popular_recs = popularity_recommendation(data_train, n=5)

result['popular_recommendation'] = result['user_id'].apply(lambda x: popular_recs)

CPU times: user 292 ms, sys: 4.68 ms, total: 297 ms
Wall time: 734 ms


### Задание 1. Weighted Random Recommendation
Напишите код для случайных рекоммендаций, в которых вероятность рекомендовать товар прямо пропорциональна логарифму продаж

 - Можно сэмплировать товары случайно, но пропорционально какому-либо весу
 - Например, прямопропорционально популярности. Вес = log(sales_sum товара)

In [206]:
def calc_log_item_weights(data):
    item_weights = data.groupby(by='item_id')['sales_value'] \
       .sum() \
       .reset_index() \
       .rename(columns={'sales_value':'weight'})
    
    item_weights['weight'] += 1
    item_weights['weight'] = np.log(item_weights['weight'])
    item_weights['weight'] /= item_weights['weight'].sum()
    
    return item_weights

In [207]:
log_item_weights = calc_log_item_weights(data)

In [208]:
def weighted_random_recommendation(items_weights, n=5):
    """Случайные рекоммендации
    
    Input
    -----
    items_weights: pd.DataFrame
        Датафрейм со столбцами item_id, weight. Сумма weight по всем товарам = 1
    """
    
    # Подсказка: необходимо модифицировать функцию random_recommendation()
    items = np.array(items_weights['item_id'])
    recs = np.random.choice(items, size=n, replace=False, p=items_weights['weight'])
    
    return recs.tolist()

In [209]:
%%time

result['random_recommendation_log_weight'] = result['user_id'].apply(lambda x: weighted_random_recommendation(log_item_weights))
# result.drop('weighted_random_recommendation', axis =1 , inplace = True)

CPU times: user 4.57 s, sys: 211 ms, total: 4.79 s
Wall time: 7.89 s


In [210]:
result.head()

,user_id,actual,random_recommendation,popular_recommendation,random_recommendation_log_weight,itemitem,cosine,own_purchases,tfidf
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[12605893, 15568311, 1091901, 1020768, 520897]","[6534178, 6533889, 1029743, 6534166, 1082185]","[8157388, 1499438, 945979, 1063207, 1041799]","[999999, 1082185, 981760, 1127831, 995242]","[1082185, 999999, 981760, 1127831, 1098066]","[1082185, 1029743, 995785, 1004906, 1081177]","[1082185, 981760, 1127831, 999999, 1098066]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[2427321, 14050273, 1703504, 5569198, 12739247]","[6534178, 6533889, 1029743, 6534166, 1082185]","[960225, 990256, 843929, 13777297, 9879531]","[999999, 1082185, 981760, 1098066, 995242]","[1082185, 1098066, 981760, 999999, 826249]","[1082185, 1098066, 6534178, 1127831, 1068719]","[1082185, 981760, 1098066, 826249, 999999]"
2,6,"[920308, 926804, 946489, 1006718, 1017061, 107...","[887953, 9296891, 6469806, 1096317, 13008170]","[6534178, 6533889, 1029743, 6534166, 1082185]","[13003094, 975508, 882787, 1185559, 1085658]","[999999, 1082185, 981760, 1127831, 995242]","[1082185, 999999, 981760, 1127831, 1098066]","[1082185, 1029743, 6534178, 1127831, 995785]","[1082185, 981760, 1127831, 999999, 878996]"
3,7,"[840386, 889774, 898068, 909714, 929067, 95347...","[845636, 1020716, 8018966, 1740284, 1602302]","[6534178, 6533889, 1029743, 6534166, 1082185]","[6772819, 1003257, 13095425, 1099192, 12301177]","[999999, 1082185, 981760, 1127831, 995242]","[1082185, 981760, 999999, 1127831, 961554]","[1082185, 1029743, 1127831, 995785, 1044078]","[1082185, 981760, 1127831, 999999, 961554]"
4,8,"[835098, 872137, 910439, 924610, 992977, 10412...","[1086732, 12810575, 12351900, 1132295, 14025586]","[6534178, 6533889, 1029743, 6534166, 1082185]","[933165, 12487931, 942935, 74379, 1078833]","[999999, 1082185, 981760, 1127831, 1098066]","[1082185, 981760, 999999, 1098066, 826249]","[1082185, 1029743, 1098066, 6534178, 1127831]","[1082185, 981760, 999999, 1098066, 826249]"


### Задание 2. Расчет метрик
Рассчитайте Precision@5 для каждого алгоритма с помощью функции из вебинара 1. Какой алгоритм показывает лучшее качество?

### Item-Item Recommender / ItemKNN

In [211]:
popularity = data_train.groupby('item_id')['quantity'].sum().reset_index()
popularity.rename(columns={'quantity': 'n_sold'}, inplace=True)

popularity.head()

,item_id,n_sold
0,25671,6
1,26081,1
2,26093,1
3,26190,1
4,26355,2


In [212]:
top_5000 = popularity.sort_values('n_sold', ascending=False).head(5000).item_id.tolist()

In [213]:
data_train.loc[~data_train['item_id'].isin(top_5000), 'item_id'] = 999999
data_train.head()

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1763: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.60,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.00,1631,1,0.0,0.0
2,2375,26984851472,1,1036325,1,0.99,364,-0.30,1631,1,0.0,0.0
3,2375,26984851472,1,1082185,1,1.21,364,0.00,1631,1,0.0,0.0
4,2375,26984851472,1,8160430,1,1.50,364,-0.39,1631,1,0.0,0.0


In [214]:
user_item_matrix = pd.pivot_table(data_train, 
                                  index='user_id', columns='item_id', 
                                  values='quantity',
                                  aggfunc='count', 
                                  fill_value=0
                                 )

user_item_matrix[user_item_matrix > 0] = 1 # так как в итоге хотим предсказать 
user_item_matrix = user_item_matrix.astype(float) # необходимый тип матрицы для implicit

# переведем в формат sparse matrix
sparse_user_item = csr_matrix(user_item_matrix).tocsr()

In [215]:
userids = user_item_matrix.index.values
itemids = user_item_matrix.columns.values

matrix_userids = np.arange(len(userids))
matrix_itemids = np.arange(len(itemids))

id_to_itemid = dict(zip(matrix_itemids, itemids))
id_to_userid = dict(zip(matrix_userids, userids))

itemid_to_id = dict(zip(itemids, matrix_itemids))
userid_to_id = dict(zip(userids, matrix_userids))

In [216]:
%%time

model = ItemItemRecommender(K=5, num_threads=4) # K - кол-во билжайших соседей


model.fit(csr_matrix(user_item_matrix).T.tocsr(),  # На вход item-user matrix
          show_progress=True)


recs = model.recommend(userid=userid_to_id[2],  # userid - id от 0 до N
                        user_items=csr_matrix(user_item_matrix).tocsr(),   # на вход user-item matrix
                        N=5, # кол-во рекомендаций 
                        filter_already_liked_items=False, 
                        filter_items=None, 
                        recalculate_user=True)

  0%|          | 0/5001 [00:00<?, ?it/s]

CPU times: user 2.8 s, sys: 25.9 ms, total: 2.83 s
Wall time: 2.18 s


In [217]:
%%time

result['itemitem'] = result['user_id'].apply(lambda x: [id_to_itemid[rec[0]] for rec in 
                                    model.recommend(userid=userid_to_id[x], 
                                    user_items=sparse_user_item,   # на вход user-item matrix
                                    N=5, 
                                    filter_already_liked_items=False, 
                                    filter_items=None, 
                                    recalculate_user=True)])

CPU times: user 67.8 ms, sys: 1.87 ms, total: 69.7 ms
Wall time: 70.6 ms


In [218]:
result.head()

,user_id,actual,random_recommendation,popular_recommendation,random_recommendation_log_weight,itemitem,cosine,own_purchases,tfidf
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[12605893, 15568311, 1091901, 1020768, 520897]","[6534178, 6533889, 1029743, 6534166, 1082185]","[8157388, 1499438, 945979, 1063207, 1041799]","[999999, 1082185, 981760, 1127831, 995242]","[1082185, 999999, 981760, 1127831, 1098066]","[1082185, 1029743, 995785, 1004906, 1081177]","[1082185, 981760, 1127831, 999999, 1098066]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[2427321, 14050273, 1703504, 5569198, 12739247]","[6534178, 6533889, 1029743, 6534166, 1082185]","[960225, 990256, 843929, 13777297, 9879531]","[999999, 1082185, 981760, 1098066, 995242]","[1082185, 1098066, 981760, 999999, 826249]","[1082185, 1098066, 6534178, 1127831, 1068719]","[1082185, 981760, 1098066, 826249, 999999]"
2,6,"[920308, 926804, 946489, 1006718, 1017061, 107...","[887953, 9296891, 6469806, 1096317, 13008170]","[6534178, 6533889, 1029743, 6534166, 1082185]","[13003094, 975508, 882787, 1185559, 1085658]","[999999, 1082185, 981760, 1127831, 995242]","[1082185, 999999, 981760, 1127831, 1098066]","[1082185, 1029743, 6534178, 1127831, 995785]","[1082185, 981760, 1127831, 999999, 878996]"
3,7,"[840386, 889774, 898068, 909714, 929067, 95347...","[845636, 1020716, 8018966, 1740284, 1602302]","[6534178, 6533889, 1029743, 6534166, 1082185]","[6772819, 1003257, 13095425, 1099192, 12301177]","[999999, 1082185, 981760, 1127831, 995242]","[1082185, 981760, 999999, 1127831, 961554]","[1082185, 1029743, 1127831, 995785, 1044078]","[1082185, 981760, 1127831, 999999, 961554]"
4,8,"[835098, 872137, 910439, 924610, 992977, 10412...","[1086732, 12810575, 12351900, 1132295, 14025586]","[6534178, 6533889, 1029743, 6534166, 1082185]","[933165, 12487931, 942935, 74379, 1078833]","[999999, 1082185, 981760, 1127831, 1098066]","[1082185, 981760, 999999, 1098066, 826249]","[1082185, 1029743, 1098066, 6534178, 1127831]","[1082185, 981760, 999999, 1098066, 826249]"


### Косинусное сходство и CosineRecommender

In [219]:
%%time

model = CosineRecommender(K=5, num_threads=4) # K - кол-во билжайших соседей

model.fit(csr_matrix(user_item_matrix).T.tocsr(), 
          show_progress=True)

recs = model.recommend(userid=userid_to_id[1], 
                        user_items=csr_matrix(user_item_matrix).tocsr(),   # на вход user-item matrix
                        N=5, 
                        filter_already_liked_items=False, 
                        filter_items=None, 
                        recalculate_user=False)

  0%|          | 0/5001 [00:00<?, ?it/s]

CPU times: user 2.8 s, sys: 30.9 ms, total: 2.83 s
Wall time: 2.11 s


In [220]:
%%time

result['cosine'] = result['user_id'].\
    apply(lambda x: [id_to_itemid[rec[0]] for rec in 
                    model.recommend(userid=userid_to_id[x], 
                                    user_items=sparse_user_item,   # на вход user-item matrix
                                    N=5, 
                                    filter_already_liked_items=False, 
                                    filter_items=None, 
                                    recalculate_user=True)])

CPU times: user 80.9 ms, sys: 969 µs, total: 81.9 ms
Wall time: 82.5 ms


In [221]:
result.head()

,user_id,actual,random_recommendation,popular_recommendation,random_recommendation_log_weight,itemitem,cosine,own_purchases,tfidf
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[12605893, 15568311, 1091901, 1020768, 520897]","[6534178, 6533889, 1029743, 6534166, 1082185]","[8157388, 1499438, 945979, 1063207, 1041799]","[999999, 1082185, 981760, 1127831, 995242]","[1082185, 999999, 981760, 1127831, 1098066]","[1082185, 1029743, 995785, 1004906, 1081177]","[1082185, 981760, 1127831, 999999, 1098066]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[2427321, 14050273, 1703504, 5569198, 12739247]","[6534178, 6533889, 1029743, 6534166, 1082185]","[960225, 990256, 843929, 13777297, 9879531]","[999999, 1082185, 981760, 1098066, 995242]","[1082185, 1098066, 981760, 999999, 826249]","[1082185, 1098066, 6534178, 1127831, 1068719]","[1082185, 981760, 1098066, 826249, 999999]"
2,6,"[920308, 926804, 946489, 1006718, 1017061, 107...","[887953, 9296891, 6469806, 1096317, 13008170]","[6534178, 6533889, 1029743, 6534166, 1082185]","[13003094, 975508, 882787, 1185559, 1085658]","[999999, 1082185, 981760, 1127831, 995242]","[1082185, 999999, 981760, 1127831, 1098066]","[1082185, 1029743, 6534178, 1127831, 995785]","[1082185, 981760, 1127831, 999999, 878996]"
3,7,"[840386, 889774, 898068, 909714, 929067, 95347...","[845636, 1020716, 8018966, 1740284, 1602302]","[6534178, 6533889, 1029743, 6534166, 1082185]","[6772819, 1003257, 13095425, 1099192, 12301177]","[999999, 1082185, 981760, 1127831, 995242]","[1082185, 981760, 999999, 1127831, 961554]","[1082185, 1029743, 1127831, 995785, 1044078]","[1082185, 981760, 1127831, 999999, 961554]"
4,8,"[835098, 872137, 910439, 924610, 992977, 10412...","[1086732, 12810575, 12351900, 1132295, 14025586]","[6534178, 6533889, 1029743, 6534166, 1082185]","[933165, 12487931, 942935, 74379, 1078833]","[999999, 1082185, 981760, 1127831, 1098066]","[1082185, 981760, 999999, 1098066, 826249]","[1082185, 1029743, 1098066, 6534178, 1127831]","[1082185, 981760, 999999, 1098066, 826249]"


In [222]:
%%time

model = TFIDFRecommender(K=5, num_threads=4) # K - кол-во билжайших соседей

model.fit(csr_matrix(user_item_matrix).T.tocsr(), 
          show_progress=True)

recs = model.recommend(userid=userid_to_id[1], 
                        user_items=csr_matrix(user_item_matrix).tocsr(),   # на вход user-item matrix
                        N=5, 
                        filter_already_liked_items=False, 
                        filter_items=None, 
                        recalculate_user=False)

  0%|          | 0/5001 [00:00<?, ?it/s]

CPU times: user 2.8 s, sys: 30.1 ms, total: 2.83 s
Wall time: 2.12 s


In [223]:
%%time

result['tfidf'] = result['user_id'].\
    apply(lambda x: [id_to_itemid[rec[0]] for rec in 
                    model.recommend(userid=userid_to_id[x], 
                                    user_items=sparse_user_item,   # на вход user-item matrix
                                    N=5, 
                                    filter_already_liked_items=False, 
                                    filter_items=None, 
                                    recalculate_user=False)])

CPU times: user 84 ms, sys: 0 ns, total: 84 ms
Wall time: 85.6 ms


In [224]:
result.head()

,user_id,actual,random_recommendation,popular_recommendation,random_recommendation_log_weight,itemitem,cosine,own_purchases,tfidf
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[12605893, 15568311, 1091901, 1020768, 520897]","[6534178, 6533889, 1029743, 6534166, 1082185]","[8157388, 1499438, 945979, 1063207, 1041799]","[999999, 1082185, 981760, 1127831, 995242]","[1082185, 999999, 981760, 1127831, 1098066]","[1082185, 1029743, 995785, 1004906, 1081177]","[1082185, 981760, 1127831, 999999, 1098066]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[2427321, 14050273, 1703504, 5569198, 12739247]","[6534178, 6533889, 1029743, 6534166, 1082185]","[960225, 990256, 843929, 13777297, 9879531]","[999999, 1082185, 981760, 1098066, 995242]","[1082185, 1098066, 981760, 999999, 826249]","[1082185, 1098066, 6534178, 1127831, 1068719]","[1082185, 981760, 1098066, 826249, 999999]"
2,6,"[920308, 926804, 946489, 1006718, 1017061, 107...","[887953, 9296891, 6469806, 1096317, 13008170]","[6534178, 6533889, 1029743, 6534166, 1082185]","[13003094, 975508, 882787, 1185559, 1085658]","[999999, 1082185, 981760, 1127831, 995242]","[1082185, 999999, 981760, 1127831, 1098066]","[1082185, 1029743, 6534178, 1127831, 995785]","[1082185, 981760, 1127831, 999999, 878996]"
3,7,"[840386, 889774, 898068, 909714, 929067, 95347...","[845636, 1020716, 8018966, 1740284, 1602302]","[6534178, 6533889, 1029743, 6534166, 1082185]","[6772819, 1003257, 13095425, 1099192, 12301177]","[999999, 1082185, 981760, 1127831, 995242]","[1082185, 981760, 999999, 1127831, 961554]","[1082185, 1029743, 1127831, 995785, 1044078]","[1082185, 981760, 1127831, 999999, 961554]"
4,8,"[835098, 872137, 910439, 924610, 992977, 10412...","[1086732, 12810575, 12351900, 1132295, 14025586]","[6534178, 6533889, 1029743, 6534166, 1082185]","[933165, 12487931, 942935, 74379, 1078833]","[999999, 1082185, 981760, 1127831, 1098066]","[1082185, 981760, 999999, 1098066, 826249]","[1082185, 1029743, 1098066, 6534178, 1127831]","[1082185, 981760, 999999, 1098066, 826249]"


In [225]:
def precision_at_k(recommended_list, bought_list, k=5):
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)[:k]
    flags = np.isin(bought_list, recommended_list)
    precision = flags.sum() / len(recommended_list)

    return precision

In [226]:
model_names = [m for m in list(result)[2:] ]

In [227]:
model_names

['random_recommendation',
 'popular_recommendation',
 'random_recommendation_log_weight',
 'itemitem',
 'cosine',
 'own_purchases',
 'tfidf']

In [228]:
k = 5
pd.DataFrame(
    [
        (model_name, result.apply(lambda row: precision_at_k(recommended_list=row[model_name], 
                                                             bought_list=row['actual'], 
                                                             k=k), 
                                                             axis=1).mean())
        for model_name in model_names
    ], 
    columns=['model name', f'precision@{k}']
)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: RuntimeWarning: invalid value encountered in long_scalars
  """


,model name,precision@5
0,random_recommendation,0.000588
1,popular_recommendation,0.155240
2,random_recommendation_log_weight,0.001763
3,itemitem,0.136925
4,cosine,0.132909
5,own_purchases,0.201917
6,tfidf,0.138981


### Задание 3*. Улучшение бейзлайнов и ItemItem

- Попробуйте улучшить бейзлайны, считая их на топ-5000 товаров
- Попробуйте улучшить разные варианты ItemItemRecommender, выбирая число соседей $K$.